# STEM-DPC dataprosessering: notebook 2

Denne Jupyter Notebook bruker `..._bs.hspy` signalet (beam shift), og `..._vbf.hspy` signalet (virtual bright field) til å fjerne effekten av "dscan". Ergo en slags bakgrunnsfjerning.

Måten dette gjøres er ved at man tilpasser to plan til beam shift signalet. Et for x-komponenten, og et for y-komponenten. Dette skal man helst gjøre på et område som ikke er magnetisk.

For disse datasetten, så bruker vi området som har blitt bombardert med Ga-ioner. Selv om det er mye signal fra disse områdene, så er det primært ikke-magnetisk. Ergo virker det bra for å tilpasse de to planene. Dette gjør vi ved å bruke VBF-signalet:

- Først kjører vi "otsu-threshold" på VBF-signalet:  `skimage.filters.threshold_otsu`
- Så fjerner vi en god del av de små variasjonene: `skimage.morphology.binary_dilation`, og "åpner" litt opp med ``skimage.morphology.binary_erosion`
- Så får vi en maske, som vi bruker til å finne de to planene
- Tilslutt så fjerner vi "bakgrunnen" i `..._bs.hspy`, og får et korrigert beam shift signal.

Deretter lager den 6 signaler i `002_cor_data`:

- "Bakgrunnsfjernet" beam shift: `...bs_cor.hspy`
- Virtual bright field: `..._vbf.hspy`
- x-komponenten av beam shift: `...bs_dpcx.hspy`
- y-komponenten av beam shift: `...bs_dpcy.hspy`
- Styrken av magnetfeltet: `...bs_mag.hspy`
- Masken som blir lagd: `...vbf_mask.hspy`

Dette blir brukt i `003_plotte_data.ipynb` for å lage en figur for å sjekke at dataprosesseringen gikk greit. `003_plotte_data.ipynb` har også tips om hvordan dere kan visualisere dataene deres.

In [12]:
%matplotlib inline
from glob import glob
import numpy as np
from skimage.filters import threshold_otsu
from skimage.morphology import binary_dilation, binary_erosion, area_closing
import hyperspy.api as hs
import gc

filename_list_vbf = glob("001_data/*_vbf.hspy")
filename_list_bs = glob("001_data/*_bs.hspy")

# Prosessere og lagre data

In [13]:
#### From magnus ####
for filename_vbf, filename_bs in zip(filename_list_vbf, filename_list_bs):
    s_vbf = hs.load(filename_vbf)
    s_t = threshold_otsu(s_vbf.data)
    s_mask = s_vbf > s_t
    s_mask_1 = s_mask.map(binary_dilation, inplace=False, show_progressbar=False)
    s_mask_2 = s_mask_1.map(binary_dilation, inplace=False, show_progressbar=False)
    s_mask_3 = s_mask_2.map(binary_dilation, inplace=False, show_progressbar=False)
    s_mask_4 = s_mask_3.map(binary_erosion, inplace=False, show_progressbar=False)
    s_mask_5 = s_mask_4.map(binary_erosion, inplace=False, show_progressbar=False)
    s_mask_5.data = np.invert(s_mask_5.data)
    
    s_bs = hs.load(filename_bs)
    s_bs_lp = s_bs.get_linear_plane(mask=s_mask_5)
    s_bs_cor = s_bs - s_bs_lp
    filename_output = filename_bs.replace("001_data", "002_cor_data").replace(".hspy", "_cor.hspy")
    filename_vbf = filename_vbf.replace("001_data", "002_cor_data")
    filename_dpcx = filename_bs.replace("001_data", "002_cor_data").replace(".hspy", "_dpcx.hspy")
    filename_dpcy = filename_bs.replace("001_data", "002_cor_data").replace(".hspy", "_dpcy.hspy")
    filename_mag = filename_bs.replace("001_data", "002_cor_data").replace(".hspy", "_mag.hspy")
    filename_mask_output = filename_vbf.replace("001_data", "002_cor_data").replace(".hspy", "_mask.hspy")
    s_bs_cor.save(filename_output, overwrite=True)
    s_vbf.save(filename_vbf, overwrite=True)
    s_bs_cor.isig[0].T.save(filename_dpcx, overwrite=True)
    s_bs_cor.isig[1].T.save(filename_dpcy, overwrite=True)
    s_bs_cor.get_magnitude_signal().save(filename_mag, overwrite=True)
    s_mask_5.save(filename_mask_output, overwrite=True)

[########################################] | 100% Completed | 233.89 ms
[########################################] | 100% Completed | 280.56 ms
[########################################] | 100% Completed | 277.50 ms
[########################################] | 100% Completed | 283.59 ms
[########################################] | 100% Completed | 248.16 ms
[########################################] | 100% Completed | 276.10 ms
[########################################] | 100% Completed | 283.19 ms
[########################################] | 100% Completed | 276.81 ms
[########################################] | 100% Completed | 273.67 ms
[########################################] | 100% Completed | 229.44 ms
[########################################] | 100% Completed | 283.08 ms
[########################################] | 100% Completed | 277.90 ms
[########################################] | 100% Completed | 244.51 ms
[########################################] | 100% Completed | 27

In [14]:
### Alternative from Jens ###

number_of_binary_loops = 10

for filename_vbf, filename_bs in zip(filename_list_vbf, filename_list_bs):
    print('looking at: ', filename_bs)
    s_vbf = hs.load(filename_vbf)
    s_t = threshold_otsu(s_vbf.data)
    s_mask = s_vbf > s_t

    s_tmp1 = s_mask

    s_tmp2 = 0

    for i in range(number_of_binary_loops):
        del s_tmp2
        gc.collect()

        s_tmp2 = s_tmp1.map(binary_dilation, inplace=False, show_progressbar=False)

        del s_tmp1
        gc.collect()

        s_tmp1 = s_tmp2.map(binary_erosion, inplace=False, show_progressbar=False)

    s_mask_5 = s_tmp1
    
    
    
    s_mask_5.data = np.invert(s_mask_5.data)

    s_bs = hs.load(filename_bs)
    s_bs_lp = s_bs.get_linear_plane(mask=s_mask_5)
    s_bs_cor = s_bs - s_bs_lp
    filename_output = filename_bs.replace("001_data", "002_cor_data_more").replace(".hspy", "_cor_more.hspy")
    filename_vbf = filename_vbf.replace("001_data", "002_cor_data_more")
    filename_dpcx = filename_bs.replace("001_data", "002_cor_data_more").replace(".hspy", "_dpcx_more.hspy")
    filename_dpcy = filename_bs.replace("001_data", "002_cor_data_more").replace(".hspy", "_dpcy_more.hspy")
    filename_mag = filename_bs.replace("001_data", "002_cor_data_more").replace(".hspy", "_mag_more.hspy")
    filename_mask_output = filename_vbf.replace("001_data", "002_cor_data").replace(".hspy", "_mask_more.hspy")
    s_bs_cor.save(filename_output, overwrite=True)
    s_vbf.save(filename_vbf, overwrite=True)
    s_bs_cor.isig[0].T.save(filename_dpcx, overwrite=True)
    s_bs_cor.isig[1].T.save(filename_dpcy, overwrite=True)
    s_bs_cor.get_magnitude_signal().save(filename_mag, overwrite=True)
    s_mask_5.save(filename_mask_output, overwrite=True)

looking at:  001_data/Av5_deg4.1_bs.hspy
[########################################] | 100% Completed | 291.42 ms
looking at:  001_data/kompass_deg-10.2_bs.hspy
[########################################] | 100% Completed | 240.11 ms
looking at:  001_data/Av5_deg9.8_bs.hspy
[########################################] | 100% Completed | 277.79 ms
looking at:  001_data/Ev5_deg9.9_bs.hspy
[########################################] | 100% Completed | 276.27 ms
looking at:  001_data/Bv5_deg10.1_bs.hspy
[########################################] | 100% Completed | 273.82 ms
looking at:  001_data/kompass_deg5_bs.hspy
[########################################] | 100% Completed | 479.48 ms
looking at:  001_data/Ev5_deg-0.3_bs.hspy
[########################################] | 100% Completed | 416.35 ms
looking at:  001_data/structure1neg5deg_bs.hspy
[########################################] | 100% Completed | 275.42 ms
looking at:  001_data/Bv5_deg0.1_bs.hspy
[#####################################